In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn.apionly as sns
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV,KFold,train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE,SelectKBest
from sklearn.preprocessing import PolynomialFeatures,Imputer,MinMaxScaler
%matplotlib inline
sns.set_context('talk')
sns.set_style('dark')

In [424]:
# Carrega os dados
happy_df = pd.read_csv('MassaDeDados - Use Esta Para Treinar.csv',';')
happy_df.head()

,UserID,YOB,Gender,Income,HouseholdStatus,EducationLevel,Party,Happy,Q124742,Q124122,...,Q100010,Q99716,Q99581,Q99480,Q98869,Q98578,Q98059,Q98078,Q98197,Q96024
0,2,1985.0,Female,"$25,001 - $50,000",Single (no kids),Master's Degree,Democrat,1,NaN,Yes,...,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN
1,5,1963.0,Male,"over $150,000",Married (w/kids),NaN,NaN,0,No,Yes,...,Yes,No,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes
2,8,1991.0,Female,"under $25,000",Single (no kids),Current Undergraduate,NaN,1,Yes,Yes,...,No,No,No,Yes,No,NaN,Yes,No,NaN,NaN
3,9,1995.0,Male,"$75,000 - $100,000",Single (no kids),Current K-12,Republican,1,NaN,NaN,...,No,No,No,Yes,Yes,No,Yes,No,Yes,Yes
4,13,1997.0,Female,"over $150,000",Single (no kids),Current K-12,Democrat,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [426]:
# Completa os dados faltantes com os valores mais frequentes 
columns = happy_df.drop(['Happy','UserID'],axis=1).columns

for col in columns:
    value_counts = happy_df[col].value_counts()
    happy_df[col].fillna(value=np.argmax(value_counts),inplace=True)  


In [427]:
# Transforma as features categóricas em binárias

cats = happy_df.drop(['YOB','Happy','UserID'],axis=1).columns
dummy_df = pd.get_dummies(columns=cats,data=happy_df,drop_first=True)
dummy_df.shape

(1528, 125)

In [471]:
# Divide os dados em treino e teste

y = dummy_df['Happy']
X = dummy_df.drop(['Happy','UserID'],axis=1)
np.random.seed(9001)
xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=0.7)

# Normaliza os dados

scaler = MinMaxScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xtest_scaled = scaler.transform(xtest)


In [475]:
# Verifica qual o número de features que melhor explica os dados através de análise de variância

scores = []
for k in range(1,xtrain.shape[1]):
    #divide os dados em treino e validação
    selector = SelectKBest(k=k) # Instancia um selector de features
    xtraink = selector.fit_transform(xtrain_scaled,ytrain) # Seleciona as k features no data set de treino
    xtestk = selector.transform(xtest_scaled)# Seleciona as k features no data set de treino
    l = LogisticRegression() # Instancia um clf LogReg
    l.fit(xtraink,ytrain) # Treina o classificador
    score = l.score(X=xtestk,y=ytest) # Calcula a acurácia
    scores.append(score)
        

In [483]:
# Seleciona as melhores features
best_k = np.argmax(scores)
selector = SelectKBest(k=best_k)
xtraink = selector.fit_transform(xtrain_scaled,ytrain) # 
xtestk = selector.transform(xtest_scaled)

# Aplica PCA
pca = PCA(n_components=17)
pca.fit(xtrain_scaled)
xtrain_pca = pca.transform(xtrain_scaled)
xtest_pca = pca.transform(xtest_scaled)
sum(pca.explained_variance_ratio_) #TODO: fazer um plote da VE

0.36674300558473583

In [477]:
print('O número de features ideal é :',best_k)

O número de features ideal é : 17


In [478]:
# Treina um classificador LogisticRegression com as features selecionadas

lr = LogisticRegressionCV()#Instacia uma Regressão Logística
print('Treinando uma Regressão Logística com os preditores selecionados...\n ')

# Treina o classificador e seleciona o melhor parâmetro de regularização  por crosvalidação
lr.fit(xtraink,ytrain)
print("Acurácia de treino:{:.2f}".format(lr.score(xtraink,ytrain)))
print("Acurácia de teste:{:.2f}".format(lr.score(xtraink,ytrain)))

Treinando uma Regressão Logística com os preditores selecionados...
 
Acurácia de treino:0.69
Acurácia de teste:0.69


In [485]:
# Treina um classificador LogisticRegression

lr = LogisticRegressionCV() #Instacia uma Regressão Logística
# Treina o classificador e seleciona o melhor parâmetro de regularização  por crosvalidação
lr.fit(xtrain_pca,ytrain) 
print("Acurácia de treino:{:.2f}".format(lr.score(xtrain_pca,ytrain)))
print("Acurácia de teste:{:.2f}".format(lr.score(xtrain_pca,ytrain)))


Acurácia de treino:0.67
Acurácia de teste:0.67


In [502]:
# Treina um classificador SVM com as features selecionadas

vals = [10**i for i in range(-4,3)] # Possíveis valores dos parâmetros
param_grid = {'C':vals,'gamma':vals} # Define quais os parâmetros a serem otimizados
svmk = GridSearchCV(SVC(kernel='rbf'),param_grid=param_grid,cv=10)# instancia um SVM com kernal gaussiano
print('Treinando um classicador SVM com os {} melhores preditores..\n'.format(best_k))

# Aplica crosvalidação para encontrar o melhores parâmetros e treina o classificador
svmk.fit(xtraink,ytrain)
print("Acurácia de treino:{:.2f}".format(svmk.score(xtraink,ytrain)))
print("Acurácia de teste:{:.2f}".format(svmk.score(xtraink,ytrain)))


Treinando um classicador SVM com os 17 melhores preditores..

Acurácia de treino:0.70
Acurácia de teste:0.70


In [392]:
# Treina um classificador KNN
knn = GridSearchCV(KNeighborsClassifier(),param_grid={'n_neighbors':range(1,30)},cv=5)
print('Treinando um classicador KNN com os {} melhores preditores..\n'.format(best_k))
knn.fit(xtraink,ytrain)
print("Acurácia de treino:{:.2f}".format(knn.score(xtraink,ytrain)))
print("Acurácia de teste:{:.2f}".format(knn.score(xtraink,ytrain)))


Treinando um classicador KNN com os 17 melhores preditores..

Acurácia de treino:0.70
Acurácia de teste:0.70


In [503]:
# Carrega os dados cujas classes serão preditas
happy_test = pd.read_csv('MassaDeDadosTESTE.csv',';')

In [504]:
happy_test.head()

,UserID,YOB,Gender,Income,HouseholdStatus,EducationLevel,Party,Happy,Q124742,Q124122,...,Q100010,Q99716,Q99581,Q99480,Q98869,Q98578,Q98059,Q98078,Q98197,Q96024
0,1,1938.0,Male,NaN,Married (w/kids),NaN,Independent,NaN,No,NaN,...,Yes,No,No,NaN,No,NaN,Only-child,No,No,Yes
1,6,1997.0,Male,"$75,000 - $100,000",Single (no kids),High School Diploma,Republican,NaN,NaN,Yes,...,Yes,No,No,No,Yes,No,Yes,No,Yes,No
2,7,1996.0,Male,"$50,000 - $74,999",Single (no kids),Current K-12,NaN,NaN,No,No,...,No,No,No,Yes,Yes,Yes,Yes,No,Yes,Yes
3,11,1983.0,Male,"$100,001 - $150,000",Married (w/kids),Bachelor's Degree,Independent,NaN,No,Yes,...,No,No,No,Yes,Yes,No,Yes,No,No,Yes
4,12,1984.0,Female,"$50,000 - $74,999",Married (w/kids),High School Diploma,Republican,NaN,No,Yes,...,Yes,No,No,Yes,No,No,Yes,No,No,Yes


In [505]:
# Limpa os dados

happy_test = happy_test.drop(['Happy','UserID'],axis=1)
columns = happy_test.columns

for col in columns:
    value_counts = happy_test[col].value_counts()
    happy_test[col].fillna(value=np.argmax(value_counts),inplace=True)  


dummy_test_df = pd.get_dummies(columns=cats,data=happy_test,drop_first=True)
dummy_test_scaled_df =scaler.transform(dummy_test_df)
new_df = selector.transform(dummy_test_scaled_df)

In [506]:
# Realiza as predições com o classificaor SVM treinando anteriormente
predictions = svmk.predict(new_df)

In [508]:
out = pd.DataFrame(predictions)
out.to_excel('output.xlsx')